In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer

### Cek GPU

In [5]:
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Version:", torch.version.cuda)

PyTorch Version: 2.6.0+cu124
CUDA Available: True
CUDA Version: 12.4


### Load dataset

In [6]:
train_df = pd.read_csv("./dataset/rev/cleaned-trainDataset-HgSekar.csv")
valid_df = pd.read_csv("./dataset/rev/cleaned-valDataset-chatGPT.csv")

In [7]:
train_df.head()

,comment,label
0,terima kasih shoppe paket ny udah datang denga...,negatif
1,kecewa sekali box nya kurang rapi dan produkny...,negatif
2,its been days and the itch still haven't gone ...,negatif
3,bukanny makin mulus malah tumbuh jerawat gede ...,negatif
4,pengalaman penggunaan:gampang patah,negatif


In [8]:
valid_df.head()

,No.,comment,label
0,1,Produk ini sangat bagus! Kulitku jadi lebih ha...,positif
1,2,"Aku kecewa, tidak ada perubahan setelah pemaka...",negatif
2,3,"Biasa saja, tidak terlalu buruk tapi juga tida...",netral
3,4,"Wanginya enak dan cepat meresap ke kulit, suka...",positif
4,5,Teksturnya terlalu lengket dan bikin wajahku b...,negatif


### Generate Custom Dictionary

In [23]:
import pandas as pd
import re

# 1. Siapkan kamus kata sentimen
positive_words = {"bagus", "baik", "indah", "cerah", "puas", "suka", "alhamdulillah"}
negative_words = {"buruk", "jelek", "kecewa", "mengecewakan"}
negation_words = {"tidak", "bukan", "kurang", "belum", "tak"}

def preprocess_negation(text):
    if not isinstance(text, str):  # Jika text bukan string, ubah menjadi string kosong
        text = ""
    
    words = re.findall(r'\b\w+\b', text.lower())
    new_words = []
    i = 0
    while i < len(words):
        word = words[i]
        if word in negation_words and i + 1 < len(words):
            next_word = words[i+1]
            if next_word in positive_words:
                new_words.append("NEG_" + next_word)
                i += 2
                continue
            elif next_word in negative_words:
                new_words.append("POS_" + next_word)
                i += 2
                continue
            else:
                new_words.append(word)
        else:
            new_words.append(word)
        i += 1
    return " ".join(new_words)

def sentiment_score(text):
    processed_text = preprocess_negation(text)
    score = 0
    words = processed_text.split()
    for word in words:
        if word in positive_words:
            score += 1
        elif word in negative_words:
            score -= 1
        elif word.startswith("NEG_") and word[4:] in positive_words:
            score -= 1
        elif word.startswith("POS_") and word[4:] in negative_words:
            score += 1
    if score > 0:
        return "positif"
    elif score < 0:
        return "negatif"
    else:
        return "netral"

# 4. Membaca file CSV
df = pd.read_csv("./test_comments_with_predictions.csv")
# 5. Hapus baris dengan nilai NaN pada kolom "comment"
df = df.dropna(subset=["comment"])

# 6. Terapkan analisis sentimen ke setiap komentar
df["sentiment"] = df["comment"].apply(sentiment_score)

# 7. Simpan hasil ke file baru
df.to_csv("classified_comments.csv", index=False)

# 8. Tampilkan hasil
print(df)


       no                                            comment predicted_label  \
0       1  Aku cuma pake sunscreen dan krim ini doang, al...         positif   
1       2  Simpen dulu di keranjang kuning, nanti check out.         negatif   
2       3  Muka kusamku jadi segeran setelah rutin pake k...         positif   
3       4  Aku cuma pake sunscreen dan krim ini doang, al...         positif   
4       5  Masih menjadi misteri kenapa Dr. Fay bisa seba...         negatif   
..    ...                                                ...             ...   
995   996  Pengiriman cepet juga ya, ke Bekasi 2 hari sampe.         positif   
996   997  Akhirnya nemu skincare yang cocok dan gak biki...         positif   
997   998  Ternyata gak kerasa, belum sebulan pake udah a...          netral   
998   999  Yang mau hasilnya maksimal, pake rutin tiap ma...         positif   
999  1000  Gak kecewa sama sekali beli ini, cuma pengirim...         positif   

    sentiment  
0     positif  
1      

In [10]:
train_df['customPred'] = train_df['comment'].apply(sentiment_score)
accuracy = (train_df['customPred'] == train_df['label']).mean()
print(f"Baseline Accuracy: {accuracy:.4f}")

Baseline Accuracy: 0.3445


In [11]:
valid_df['customPred'] = valid_df['comment'].apply(sentiment_score)
accuracy = (valid_df['customPred'] == valid_df['label']).mean()
print(f"Baseline Accuracy: {accuracy:.4f}")


Baseline Accuracy: 0.4875


### Tokenization

In [12]:
from transformers import BertTokenizer

# Gunakan BertTokenizer untuk IndoBERT
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p1")

In [20]:
# Mengubah teks customPredict menjadi token
train_df['customPred_num'] = train_df['comment'].apply(lambda x: 2 if sentiment_score(x) == "positif" else (0 if sentiment_score(x) == "negatif" else 1))
valid_df['customPred_num'] = valid_df['comment'].apply(lambda x: 2 if sentiment_score(x) == "positif" else (0 if sentiment_score(x) == "negatif" else 1))

In [21]:
# mengubah teks label menjadi token
label_mapping = {'negatif': 0, 'netral': 1,'positif': 2}
train_df['label_num'] = train_df['label'].map(label_mapping)
valid_df['label_num'] = valid_df['label'].map(label_mapping)

In [22]:
train_df.head()

,No.,comment,label,customPred,customPred_num,label_num
0,1,produk ini sangat bagus kulitku jadi lebih hal...,positif,positif,2,2
1,2,aku kecewa tidak ada perubahan setelah pemakai...,negatif,negatif,0,0
2,3,biasa saja tidak terlalu buruk tapi juga tidak...,netral,positif,2,1
3,4,wanginya enak dan cepat meresap ke kulit suka ...,positif,positif,2,2
4,5,teksturnya terlalu lengket dan bikin wajahku b...,negatif,netral,1,0


In [23]:
valid_df.head()

,no,comment,label,customPred,customPred_num,label_num
0,1,aku cuma pake sunscreen dan krim ini doang alh...,positif,positif,2,2
1,2,beli ini karena reviewnya bagus semoga cocok d...,netral,positif,2,1
2,3,gak nyangka produk ini bikin kulitku tambah cerah,positif,positif,2,2
3,4,udah coba pakai seminggu tapi kok belum ada pe...,netral,netral,1,1
4,5,teksturnya terlalu lengket di kulitku kurang n...,negatif,netral,1,0


In [24]:
train_df.head()

,No.,comment,label,customPred,customPred_num,label_num
0,1,produk ini sangat bagus kulitku jadi lebih hal...,positif,positif,2,2
1,2,aku kecewa tidak ada perubahan setelah pemakai...,negatif,negatif,0,0
2,3,biasa saja tidak terlalu buruk tapi juga tidak...,netral,positif,2,1
3,4,wanginya enak dan cepat meresap ke kulit suka ...,positif,positif,2,2
4,5,teksturnya terlalu lengket dan bikin wajahku b...,negatif,netral,1,0


In [25]:
valid_df.head()

,no,comment,label,customPred,customPred_num,label_num
0,1,aku cuma pake sunscreen dan krim ini doang alh...,positif,positif,2,2
1,2,beli ini karena reviewnya bagus semoga cocok d...,netral,positif,2,1
2,3,gak nyangka produk ini bikin kulitku tambah cerah,positif,positif,2,2
3,4,udah coba pakai seminggu tapi kok belum ada pe...,netral,netral,1,1
4,5,teksturnya terlalu lengket di kulitku kurang n...,negatif,netral,1,0


### Build Model

In [26]:
class IndoBERT_BiLSTM_Enhanced(nn.Module):
    def __init__(self, bert_model="indobenchmark/indobert-base-p1", lstm_hidden=64, num_classes=3):
        super(IndoBERT_BiLSTM_Enhanced, self).__init__()
        
        self.bert = AutoModel.from_pretrained(bert_model)
        self.bert.requires_grad_(False)
        
        self.lstm = nn.LSTM(input_size=768, hidden_size=lstm_hidden, num_layers=1, 
                            batch_first=True, bidirectional=True, dropout=0.2)
        
        self.fc = nn.Linear(lstm_hidden * 2 + 1, num_classes)  # +1 untuk fitur sentiment_score
        self.dropout = nn.Dropout(0.2)

    def forward(self, input_ids, attention_mask, sentiment_score):
        with torch.no_grad():
            bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        
        bert_embedding = bert_output.last_hidden_state[:, 0, :]
        lstm_out, _ = self.lstm(bert_embedding.unsqueeze(1))
        lstm_out = lstm_out[:, -1, :]
        
        lstm_out = self.dropout(lstm_out)
        
        # Gabungkan fitur sentiment_score
        combined_features = torch.cat((lstm_out, sentiment_score.unsqueeze(1)), dim=1)
        output = self.fc(combined_features)
        return output

In [27]:
train_df.head()

,No.,comment,label,customPred,customPred_num,label_num
0,1,produk ini sangat bagus kulitku jadi lebih hal...,positif,positif,2,2
1,2,aku kecewa tidak ada perubahan setelah pemakai...,negatif,negatif,0,0
2,3,biasa saja tidak terlalu buruk tapi juga tidak...,netral,positif,2,1
3,4,wanginya enak dan cepat meresap ke kulit suka ...,positif,positif,2,2
4,5,teksturnya terlalu lengket dan bikin wajahku b...,negatif,netral,1,0


In [28]:
for batch in train_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['label'].to(device)
    sentiment_scores = batch['sentiment_score'].to(device)
    
    optimizer.zero_grad()
    outputs = model(input_ids, attention_mask, sentiment_scores)
    
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

NameError: name 'train_loader' is not defined

In [24]:
sentiment_score = torch.tensor(int(score), dtype=torch.long)

NameError: name 'score' is not defined